<a href="https://colab.research.google.com/github/Eshita781996/SCD/blob/main/SCD1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
!pip -q install findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages io.delta:delta-core_2.12:0.7.0 --conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog pyspark-shell'

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('delta_session').getOrCreate()

In [ ]:
# employees_df = spark.createDataFrame([(1, 'John', 'Sydney', 35000.00),(2, 'Peter', 'Melbourne', 45000.00),(3, 'Sam', 'Sydney',55000.00)],
#                                      schema='emp_id int,emp_name string,emp_city string,emp_salary float')

employees_delta_df = spark.createDataFrame([(2, 'Peter', 'Melbourne', 55000.00),(5, 'Jessie', 'Brisbane', 42000.00)],
                                           schema='emp_id int,emp_name string,emp_city string,emp_salary float')

In [ ]:
#employees_df.write.format("delta").save("employee")

In [ ]:

from delta.tables import *
from delta.tables import *
if not DeltaTable.isDeltaTable(spark,'employee'):
  employees_delta_df.write.format("delta").save("employee")
else:
  deltaTable = DeltaTable.forPath(spark, "employee")
  deltaTable.alias("history").merge(
      employees_delta_df.alias("updates"),
      "history.emp_id = updates.emp_id").whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

In [ ]:
employees_df.show()
employees_delta_df.show()
spark.read.format("delta").load("employee").show()

+------+--------+---------+----------+
|emp_id|emp_name| emp_city|emp_salary|
+------+--------+---------+----------+
|     1|    John|   Sydney|   35000.0|
|     2|   Peter|Melbourne|   45000.0|
|     3|     Sam|   Sydney|   55000.0|
+------+--------+---------+----------+

+------+--------+---------+----------+
|emp_id|emp_name| emp_city|emp_salary|
+------+--------+---------+----------+
|     2|   Peter|Melbourne|   55000.0|
|     5|  Jessie| Brisbane|   42000.0|
+------+--------+---------+----------+

+------+--------+---------+----------+
|emp_id|emp_name| emp_city|emp_salary|
+------+--------+---------+----------+
|     5|  Jessie| Brisbane|   42000.0|
|     2|   Peter|Melbourne|   55000.0|
|     1|    John|   Sydney|   35000.0|
|     3|     Sam|   Sydney|   55000.0|
+------+--------+---------+----------+

